In [98]:
import numpy as np
import pandas as pd
import plotly.express as px
import math as mt
import scipy as sp
import streamlit as st
import altair as al
import plotly.colors as pc

Imported what i know ill need and will add as needed

In [99]:
game_data = pd.read_csv("../games.csv")

In [100]:
game_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [101]:
game_data.columns = game_data.columns.str.lower()
game_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


Changed all column names to lowercase to make them easier to work with.

In [102]:
name_missing = game_data[game_data['name'].isna()]
name_missing


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
659,NaN,GEN,1993.0,NaN,1.78,0.53,0.00,0.08,NaN,NaN,NaN
14244,NaN,GEN,1993.0,NaN,0.00,0.00,0.03,0.00,NaN,NaN,NaN


Although i will be dropping these as im not 100% sure i am fairly certain after a good amount opf research that entry 659 is 'Mortal Kombat II' as its the only game in 1993 on the GEN that sold 1.78 million copies in us and also had a healthy sale margin in the EU market and entry 14244 is "Battle Mania Daiginjou" as it was the only game on GEN in 1993 to not receive sales in NA or EU as it was not released there and had only 30000 sale sin jp. However again i will be dropping these lines as they are missing far too much information in multiple categories and we cannot retrieve its name i also believe this is why it was left unfilled.

In [103]:
game_data = game_data.dropna(subset=['name'])
game_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16713 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16444 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.5+ MB


i dropped the previously stated rows 

In [104]:
yor_missing = game_data[game_data['year_of_release'].isna()]
yor_missing.sample(20)


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
1301,Triple Play 99,PS,NaN,Sports,0.81,0.55,0.00,0.10,NaN,NaN,NaN
5326,Tropico 4,X360,NaN,Strategy,0.20,0.12,0.00,0.03,77.0,7.5,T
8260,Backbreaker,PS3,NaN,Sports,0.16,0.00,0.00,0.01,58.0,7,E
3883,Fishing Derby,2600,NaN,Sports,0.48,0.03,0.00,0.01,NaN,NaN,NaN
14685,World of Tanks,X360,NaN,Shooter,0.00,0.03,0.00,0.00,NaN,5.2,NaN
3024,"The Chronicles of Narnia: The Lion, The Witch ...",GBA,NaN,Action,0.48,0.18,0.00,0.01,66.0,6.8,E
4818,LEGO Harry Potter: Years 5-7,3DS,NaN,Action,0.18,0.19,0.00,0.03,71.0,6.5,E10+
2281,Namco Museum,XB,NaN,Misc,0.77,0.11,0.00,0.04,59.0,tbd,E
3413,Madden NFL 06,X360,NaN,Sports,0.54,0.00,0.01,0.03,74.0,4.9,E
13317,Monster Hunter Frontier Online,PS3,NaN,Role-Playing,0.00,0.00,0.05,0.00,NaN,NaN,NaN


I Went through a couple fixes to the missing values in the years_of_release column such as using the mean/median, dropping the columns, i learned KNN (K-Nearest Neighbors) but i ultimately decided to leave them their as they skewed the data too much and the information itself makes no sense to be imputed as the release date is a fact and not simply data. I am quite certain this data is missing due to human error as some of these games release years are listed in the title.

In [105]:
critic_missing = game_data[game_data['critic_score'].isna()]
critic_missing.sample(20)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
16472,Akatsuki no Amaneka to Aoi Kyojin,PSP,2010.0,Strategy,0.00,0.00,0.01,0.00,NaN,NaN,NaN
15207,Minna no Chizu 2,PSP,2007.0,Misc,0.00,0.00,0.02,0.00,NaN,NaN,NaN
10665,Evil Dead: Fistfull of Boomstick,XB,2003.0,Action,0.07,0.02,0.00,0.00,NaN,NaN,NaN
7156,Petz Monkeyz House,DS,2008.0,Simulation,0.20,0.00,0.00,0.02,NaN,tbd,E
6388,Cabela's Big Game Hunter 2010,X360,2009.0,Sports,0.25,0.00,0.00,0.02,NaN,5.6,T
12738,Katekyoo Hitman Reborn! DS Flame Rumble X - Mi...,DS,2009.0,Fighting,0.00,0.00,0.06,0.00,NaN,NaN,NaN
1343,Triple Play 2000,PS,1999.0,Sports,0.79,0.54,0.00,0.09,NaN,NaN,NaN
6996,Lunar: Silver Star Story,SAT,1996.0,Role-Playing,0.00,0.00,0.23,0.00,NaN,NaN,NaN
11449,Murder in Venice,DS,2011.0,Adventure,0.05,0.02,0.00,0.01,NaN,tbd,E10+
6387,Pinkalicious,DS,2011.0,Misc,0.25,0.00,0.00,0.02,NaN,tbd,E


In [106]:
game_data['user_score'].value_counts()

user_score
tbd    2424
7.8     324
8       290
8.2     282
8.3     254
       ... 
1.3       2
1         2
0.9       2
0         1
9.7       1
Name: count, Length: 96, dtype: int64

Noticed alot fo tbd values need to address that

In [107]:
tbd_scores = game_data[game_data['user_score'] == 'tbd']
tbd_scores

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
119,Zumba Fitness,Wii,2010.0,Sports,3.45,2.59,0.0,0.66,NaN,tbd,E
301,Namco Museum: 50th Anniversary,PS2,2005.0,Misc,2.08,1.35,0.0,0.54,61.0,tbd,E10+
520,Zumba Fitness 2,Wii,2011.0,Sports,1.51,1.03,0.0,0.27,NaN,tbd,T
645,uDraw Studio,Wii,2010.0,Misc,1.65,0.57,0.0,0.20,71.0,tbd,E
657,Frogger's Adventures: Temple of the Frog,GBA,NaN,Adventure,2.15,0.18,0.0,0.07,73.0,tbd,E
...,...,...,...,...,...,...,...,...,...,...,...
16695,Planet Monsters,GBA,2001.0,Action,0.01,0.00,0.0,0.00,67.0,tbd,E
16697,Bust-A-Move 3000,GC,2003.0,Puzzle,0.01,0.00,0.0,0.00,53.0,tbd,E
16698,Mega Brain Boost,DS,2008.0,Puzzle,0.01,0.00,0.0,0.00,48.0,tbd,E
16704,Plushees,DS,2008.0,Simulation,0.01,0.00,0.0,0.00,NaN,tbd,E


Because i could need these for future calculations i made a separate dataframe for them in case i need them grouped before i drop the tbd for a missing value

In [108]:
game_data['user_score'] = pd.to_numeric(game_data['user_score'], errors='coerce')


game_data['user_score'].value_counts()


user_score
7.8    324
8.0    290
8.2    282
8.3    254
8.5    253
      ... 
1.3      2
1.0      2
0.9      2
0.0      1
9.7      1
Name: count, Length: 95, dtype: int64

I then coerced them into numeric or 'int' which solved the data type problem of them being an object and the tbd to missing value problem

In [109]:
score_columns = ['critic_score', 'user_score', 'rating']
filtered_scores = game_data[score_columns]
filtered_scores

,critic_score,user_score,rating
0,76.0,8.0,E
1,NaN,NaN,NaN
2,82.0,8.3,E
3,80.0,8.0,E
4,NaN,NaN,NaN
...,...,...,...
16710,NaN,NaN,NaN
16711,NaN,NaN,NaN
16712,NaN,NaN,NaN
16713,NaN,NaN,NaN


In [110]:
filtered_scores_cleaned = filtered_scores.dropna(how='all')

filtered_scores_cleaned


,critic_score,user_score,rating
0,76.0,8.0,E
2,82.0,8.3,E
3,80.0,8.0,E
6,89.0,8.5,E
7,58.0,6.6,E
...,...,...,...
16698,48.0,NaN,E
16702,60.0,7.2,E10+
16704,NaN,NaN,E
16705,63.0,5.8,NaN


Then I created a separate dataframe with the critic score user score and rating in one data set because they seemed to be all missing alot of values across the board so i grouped and dropped any row that was missing all 3 and i will use this for the calculations most likely down the line i believe these were left blank as the score do not exist yet indicated by over 2000 entries being labeled 'tbd' i infer there is a certain criteria that nominates a game to be reviewed and the closer you are to that criteria the more swiftly this review is given it would explain why the entries listed as 'tbd' have a more robust well of information in the critic and ratings columns as opposed ot the ones filled with simply 'NaN'

In [111]:
sales_columns = [col for col in game_data.columns if 'sales' in col]
game_data['total_sales'] = game_data[sales_columns].sum(axis=1)

game_data.head(20)



,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E,82.54
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN,40.24
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E,35.52
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E,32.77
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN,31.38
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN,30.26
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.50,2.88,89.0,8.5,E,29.80
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E,28.91
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.70,2.24,87.0,8.4,E,28.32
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN,28.31


I created a total_sales column to for future calculations.

In [112]:
yearly_releases = game_data.groupby('year_of_release')['name'].count().reset_index()
yearly_releases.columns = ['year_of_release', 'number_of_games']

yearly_releases = yearly_releases.sort_values(by='year_of_release')

yearly_releases

,year_of_release,number_of_games
0,1980.0,9
1,1981.0,46
2,1982.0,36
3,1983.0,17
4,1984.0,14
5,1985.0,14
6,1986.0,21
7,1987.0,16
8,1988.0,15
9,1989.0,17


In [113]:
fig = px.bar(
    yearly_releases, 
    x='year_of_release', 
    y='number_of_games', 
    title='Number of Games Released Per Year',
    labels={'year_of_release': 'Year of Release', 'number_of_games': 'Number of Games'},
)

fig.update_layout(
    xaxis=dict(
        tickmode='linear',
        tick0=yearly_releases['year_of_release'].min(),
        dtick=1
    )
)

fig.show()


I made a bar graph to illustrate what the distribution of games released per year would look like visually and it shows us that most games released before 1990 were few in number by its post date standards and in this i believe its wise to run any calculations on popularity, revenue, future projections, and other relevant to todays standards calculations be done with games past this date unless clarified that something vintage is our goal. Especially if it relates to the volume of releases. I also believe this serves as a boxplots purpose as the outliers are noticeable (1980-1990).
EDIT: Upon further calculation it seems 1995 is the new latest cutoff

In [114]:
# Sorting the data for the stacked bar chart by total sales
sales_data_long_sorted = (
    game_data.groupby('platform')[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum()
    .assign(total_sales=lambda df: df.sum(axis=1))
    .sort_values(by='total_sales', ascending=False)
    .drop(columns='total_sales')
    .reset_index()
    .melt(id_vars='platform', var_name='region', value_name='sales')
)

# Stacked bar chart with regions as the legend and sorted by total sales
fig_stacked = px.bar(
    sales_data_long_sorted,
    x='platform',
    y='sales',
    color='region',
    title="Sales by Platform Across Regions (Sorted by Total Sales)",
    labels={'sales': 'Sales (in millions)', 'platform': 'Platform'},
    barmode='stack',
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_stacked.update_layout(legend_title_text='Region', xaxis_title='Platform', yaxis_title='Sales (in millions)')
fig_stacked.show()

# Total sales bar chart sorted by total sales
total_sales_data_sorted = (
    game_data.groupby('platform')['total_sales'].sum()
    .reset_index()
    .sort_values(by='total_sales', ascending=False)
)

fig_total = px.bar(
    total_sales_data_sorted,
    x='platform',
    y='total_sales',
    title="Total Sales by Platform (Sorted by Total Sales)",
    labels={'total_sales': 'Total Sales (in millions)', 'platform': 'Platform'},
    color_discrete_sequence=["#636EFA"]
)
fig_total.update_layout(legend=None, xaxis_title='Platform', yaxis_title='Total Sales (in millions)')
fig_total.show()


A comparison of sale between platforms the first bar graph is an interactive graph that shows the distribution of sales between regions and platforms and the 2nd is the same however it makes it more clear the max values and how they compare against each other

In [124]:
# Platforms by total sales
top_platforms = (
    game_data.groupby('platform')['total_sales']
    .sum()
    .sort_values(ascending=False)
    .head(20)
    .index
)

top_platforms = game_data[game_data['platform'].isin(top_10_platforms)]


# Filtering the dataset for years after 1990 and the top 10 platforms
top_platforms = top_platforms[top_platforms['year_of_release'] > 1994]

# Grouping data by year and platform for the distribution
yearly_sales_top_filtered = (
    top_platforms.groupby(['year_of_release', 'platform'])['total_sales']
    .sum()
    .reset_index()
    .sort_values(by=['year_of_release', 'platform'])
)

# Creating the distribution visualization with all years labeled
fig_yearly_distribution_filtered = px.bar(
    yearly_sales_top_filtered,
    x='year_of_release',
    y='total_sales',
    color='platform',
    title="Yearly Sales Distribution for Top 10 Platforms (1995 and Later)",
    labels={'year_of_release': 'Year of Release', 'total_sales': 'Total Sales (in millions)', 'platform': 'Platform'},
    barmode='stack',
    color_discrete_sequence=px.colors.qualitative.Vivid
)
fig_yearly_distribution_filtered.update_layout(
    legend_title_text='Platform',
    xaxis=dict(title='Year of Release', tickmode='linear', dtick=1),  # Ensuring all years are labeled
    yaxis_title='Total Sales (in millions)'
)
fig_yearly_distribution_filtered.show()


The top platforms and their sales by year in this bar graph we can see the phases in which the platforms were popular until when or if they start to be overshadowed by a new growing platform. Platforms like "PS", "PS2", "GBA", "DS", "PSP"(briefly), and "Wii" were all a top 3 seller for at least 1 year today have 0 or near 0 sales. We can use these as a good projection of the lifespan of a platform and the time intervals in between new platforms. Platforms generally have a lifespan of 7-10 years, a new platform appears generally every 5-7 years, this tracks as the time it takes for a platform to die after their appearance of another platform is roughly 3-5 years.

If our goal is the project the next couple years of we need to take data from 2006 in our above graph this is also why we have so many platforms in our dataset to better see the newer trends coming in with less values in total simply because they are newer platforms that is the also the maximum average lifespan. if current trends are to be believed then this data was most likely cutoff near the begining of 2016 thus explaining the fewer values despite consistent trends. the newer consoles ("XOne", "PS4", "WiiU") will see a decline in their current sales in the coming 2 year and spend the next 3 years fizzling out and in the next 2-3 years a new console will be introduced i believe these will be profitable as will "XOne" and "PS4" for 2016 and 2017.

In [117]:
platform_yearly_counts = game_data.groupby(['platform', 'year_of_release']).value_counts().reset_index(name='counts')

# platforms that had games released after 2013 in the dataset
platform_yearly_counts_after_2013 = platform_yearly_counts[platform_yearly_counts['year_of_release'] >= 2013]

# Unique platforms for games released after 2013
platform_yearly_counts_after_2013['platform'].unique()


array(['3DS', 'PC', 'PS3', 'PS4', 'PSP', 'PSV', 'WiiU', 'X360', 'XOne'],
      dtype=object)

So we know that 2017 is the 5th year after its 